In [12]:
import gymnasium as gym
import retro
import random
import time

from gymnasium.wrappers import TimeLimit
from stable_baselines3 import A2C, PPO
# from sbx import DDPG, DQN, PPO, SAC, TD3, TQC, CrossQ
from stable_baselines3.common.vec_env import DummyVecEnv, VecFrameStack, VecNormalize, VecTransposeImage
from stable_baselines3.common.callbacks import CheckpointCallback, BaseCallback
from stable_baselines3.common.evaluation import evaluate_policy
from gymnasium.wrappers import FrameStackObservation
from stable_baselines3.common.torch_layers import BaseFeaturesExtractor
from stable_baselines3.common.atari_wrappers import ClipRewardEnv, WarpFrame
from pathlib import Path
from sb3_contrib import RecurrentPPO
from gymnasium.wrappers import RecordVideo

import torch as th
from torch import nn
import numpy as np
import cv2
import re
import datetime

import os


LOAD_FROM="./model-sonic"
#LOAD_FROM="./model-mk3-lstm"
# LOAD_FROM="./model-stf2-features"
GAME="SonicTheHedgehog-Genesis"
STATE="GreenHillZone.Act1"
# STATE="Champion.Level1.RyuVsGuile"

states = retro.data.list_states(GAME)

# print(retro.data.list_games())
print(f"States for {GAME}: {states}")

MAX_EPISODE=5000
SAVE_VIDEO = True
IS_RECURRENT = False
USE_CURRICULUM = False
DETERMINISTIC = True
LOAD_FROM_VIDEO= LOAD_FROM + "/best_model_550000"

# LOAD_FROM="./model-sbx"
# LOAD_FROM="./model2"
# GAME="SuperMarioBros-Nes"
# STATE="Level1-1"

class IgnorePauseActionWrapper(gym.ActionWrapper):
    def __init__(self, env):
        super().__init__(env)
    
    def action(self, act):
        act[3] = 0
        return act

class Discretizer(gym.ActionWrapper):
    """
    Wrap a gym environment and make it use discrete actions.

    Args:
        combos: ordered list of lists of valid button combinations
    """

    def __init__(self, env, combos):
        super().__init__(env)
        assert isinstance(env.action_space, gym.spaces.MultiBinary)
        buttons = env.unwrapped.buttons
        self._decode_discrete_action = []
        for combo in combos:
            arr = np.array([False] * env.action_space.n)
            for button in combo:
                arr[buttons.index(button)] = True
            self._decode_discrete_action.append(arr)

        self.action_space = gym.spaces.Discrete(len(self._decode_discrete_action))

    # def action(self, act):
    #     return self._decode_discrete_action[act].copy()
    def action(self, act):
        return self._decode_discrete_action[act]

class MainDiscretizer(Discretizer):
    """
    Use Sonic-specific discrete actions
    based on https://github.com/openai/retro-baselines/blob/master/agents/sonic_util.py
    """

    def __init__(self, env):
        super().__init__(
            env=env,
            combos=[
                ["LEFT"],
                ["RIGHT"],
                ["B"],
            ]
        )


class ResetStateWrapper(gym.Wrapper):
    def __init__(self, env):
        super().__init__(env)
        
        self.env = env
        self.steps = 0
        self.x_last = 0
        self.lose_lives = False
        self.old_rings = 0
        #self.current_health = 40

    def reset(self, **kwargs):
        obs = self.env.reset(**kwargs)

        #self.x_last = self.env.unwrapped.data['x']

        self.steps = 0
        self.old_rings = 0

        #self.current_health = 40

        self.lose_lives = False

        return obs

    def step(self, action):
        obs, reward, done, trunc, info = self.env.step(action)

        self.steps += 1

        # reward = 0
        if reward > 0:
            reward = 0.1 # new score reward

        # Penalty for excessive jumping
        if action == 2:
            reward -= 0.08
        
        # Existential
        reward -= 0.05

        max_x = info['screen_x_end']
        current_x = info['x']
        current_rings = info['rings']

        if current_x > self.x_last:
            reward += (current_x / max_x) * 0.5
        elif current_x <= self.x_last:
            reward -= 0.02

        if reward > 0.9:
            print(reward)

        if self.x_last > 0:
            velocity = current_x - self.x_last

            if velocity > 4 :
                reward += 0.1
            # elif velocity == 0:
            #     print("stopped")
            
        self.x_last = current_x

        if self.steps > MAX_EPISODE:
            done = True
            reward -= 1

        # Sonic collide with obstacle
        if current_rings < self.old_rings:
            print("old rings")

        self.old_rings = current_rings

        if current_x >= max_x:
            print(current_x)
            reward += 2
            done = True

        if info['lives'] < 3:
            reward -= 1
            done = True

        

        # if self.current_health != info['health'] and info['health'] < self.current_health:
        #     reward += (info['health'] - 40) / 40 * 2
        #     self.current_health = info['health']

        # if info['lives'] < 3 or info['health'] <= 0:
        #     reward -= 1
        #     done = True 
        #     self.lose_lives = True

        # if done and not self.lose_lives:
        #     reward += 2
 
        return obs, reward, done, trunc, info


class RandomStateWrapper(gym.Wrapper):
    def __init__(self, env):
        super().__init__(env)
        
        self.env = env

    def get_random_state(self):
        """Select a random state from folder STATE_PATH"""
        STATE_PATH = "./States"
        states = [f for f in os.listdir(STATE_PATH) if f.endswith(".state")]
        if not states:
            raise FileNotFoundError("File not found!")
        c = random.choice(states)

        return os.path.abspath("./States/" + c)

    def reset(self, **kwargs):
        state = self.get_random_state()
        print(f"Loading state: {state}")
        self.env.load_state(state)

        obs = self.env.reset(**kwargs)

        return obs

    def step(self, action):
        obs, reward, done, trunc, info = self.env.step(action)
      
    
        return obs, reward, done, trunc, info


class CurriculumWrapper(gym.Wrapper):
    def __init__(self, env, required_wins=20, required_avg_reward=1.0):
        super().__init__(env)
        self.required_wins = required_wins
        self.required_avg_reward = required_avg_reward
        self.current_phase = 1
        self.total_wins = 0 
        self.rewards_list = []

    def reset(self, **kwargs):
        obs = self.env.reset(**kwargs)

        return obs

    def step(self, action):
        obs, reward, done, truncated, info = self.env.step(action)

        self.rewards_list.append(reward)

        could_to_next_stage = info["matches_won"] / self.current_phase >= 2

        if info["matches_won"] % 2 == 0 and info["matches_won"] > 0 and could_to_next_stage:
            self.total_wins += 1


        avg_reward = np.mean(self.rewards_list[-self.required_wins:]) if len(self.rewards_list) >= self.required_wins else np.mean(self.rewards_list)

        if could_to_next_stage and \
            ((info["matches_won"] % 2 == 0  and info["matches_won"] > 0) \
                 or (info["enemy_matches_won"] % 2 == 0 and info["enemy_matches_won"] > 0)) :
            print(info)
            print(f"🔥 stage {self.current_phase}! ({self.total_wins} fights win, avg rewards: {avg_reward:.2f})")
            done = True
        
        if self.total_wins >= self.required_wins and avg_reward >= self.required_avg_reward:
            self.current_phase += 1
            print(f"🔥 Going to next stage {self.current_phase}! ({self.total_wins} fights win, avg rewards: {avg_reward:.2f})")
            self.total_wins = 0
            self.rewards_list = []

        return obs, reward, done, truncated, info

def make_test_env(video_prefix):
    env = retro.make(
        game=GAME, 
        render_mode="rgb_array", # rgb_array or human
        state=STATE,
        #use_restricted_actions=retro.Actions.DISCRETE,
    )
    # env = RandomStateWrapper(env)
    # env = IgnorePauseActionWrapper(env)
    # env = DonkeyKongCustomActions(env)
    env = MainDiscretizer(env)
    env = ResetStateWrapper(env)
    env = WarpFrame(env)
    #env = ClipRewardEnv(env)

    if USE_CURRICULUM:
        env = CurriculumWrapper(env, required_wins=3, required_avg_reward=0.3)

    if video_prefix != None:
        video_prefix = video_prefix.replace(LOAD_FROM, "").replace("/", "")

    print("video_prefix:", video_prefix)
    
    if SAVE_VIDEO:
        env = RecordVideo(
            env, 
            video_folder="videos/", 
            # episode_trigger=lambda e: True, 
            episode_trigger=lambda episode_id: episode_id == 0, # record only first episode
            fps=60,
            name_prefix=f"gameplay_{video_prefix}"
        )
    env = DummyVecEnv([lambda: env])
    # env = VecNormalize(env, norm_obs=True, norm_reward=True, clip_obs=10.0)
    env = VecFrameStack(env, 4)
    #env = VecTransposeImage(env)
    # env = TimeLimit(env, max_episode_steps=MAX_EPISODE)
    return env

save_dir = Path(LOAD_FROM)


def get_latest_model(path):
    models = list(path.glob("best_model_*"))
    if not models:
        return None
    model_numbers = [int(re.search(r"best_model_(\d+)", str(m)).group(1)) for m in models]
    latest_model = max(model_numbers)
    return path / f"best_model_{latest_model}"

latest_model_path = get_latest_model(save_dir)

if SAVE_VIDEO:
    latest_model_path = LOAD_FROM_VIDEO


print(f"Loading existent model: {latest_model_path}")

# latest_model_path= LOAD_FROM + "/best_model_413000"

env = make_test_env(f"{latest_model_path}")

env.metadata['render_fps'] = 60

model = None

if IS_RECURRENT:
    model = RecurrentPPO.load(str(latest_model_path), device="cuda", verbose=0)
else:
    # model = PPO.load(str(latest_model_path), device="cuda", verbose=0)
    model = PPO.load(
        str(latest_model_path), 
        env=env, 
        verbose=0, 
        # learning_rate=LEARNING_RATE,
        # gae_lambda=GAE,
        # clip_range=CLIP_RANGE,
        # ent_coef=ENT_COEF,
        # gamma=GAMMA,
        # n_steps=N_STEPS
    )

obs = env.reset()
done = False


render_mode = "human"

if SAVE_VIDEO:
    render_mode = "rgb_array"

# obs = env.reset()
# while True:  # Testar cada botão individualmente
#     env.render()
#     # action = 0
#     # action = i
#     # print(f"Testando botão no índice {i}: {action}")
    
    
#     action = [1]

#     print(action)

#     #action = [[action]]


#     # if step % 5 == 0:
#     #     action = [ action[i] ]
#     # else:
#     #     action = [ action[0] ]

#     obs, reward, done, info = env.step(action)
#     time.sleep(0.01)

while True:
    # env.render("human") 
    env.render(render_mode) 
    action, _ = model.predict(obs, deterministic=DETERMINISTIC)

    # print(action)
    
    #print(action)
    #action = [0]
    obs, reward, done, info = env.step(action)

    #print(info)

    if done:
        if SAVE_VIDEO:
            break
        else:
            obs = env.reset()
        
env.close()

States for SonicTheHedgehog-Genesis: ['GreenHillZone.Act1', 'GreenHillZone.Act2', 'GreenHillZone.Act3', 'LabyrinthZone.Act1', 'LabyrinthZone.Act2', 'LabyrinthZone.Act3', 'MarbleZone.Act1', 'MarbleZone.Act2', 'MarbleZone.Act3', 'ScrapBrainZone.Act1', 'ScrapBrainZone.Act2', 'SpringYardZone.Act1', 'SpringYardZone.Act2', 'SpringYardZone.Act3', 'StarLightZone.Act1', 'StarLightZone.Act2', 'StarLightZone.Act3']
Loading existent model: ./model-sonic/best_model_550000
video_prefix: best_model_550000
old rings
